In [4]:
import importlib.util
import os
import json 
import pandas as pd 

print("--- Script: run_schwab_analysis.py (Schwab Analysis v4) ---")
print("--- Chunk 1: Setup and Module Import ---")

script_filename = 'schwab_data_analyzer.py' 
module_name_in_code = 'sda' 
sda_module = None

try:
    script_path = os.path.join(os.getcwd(), script_filename)
    if not os.path.exists(script_path):
        raise FileNotFoundError(f"The module script '{script_path}' was not found. Please ensure it's in the same directory as this driver script.")
        
    spec = importlib.util.spec_from_file_location(module_name_in_code, script_path)
    if spec is None:
        raise ImportError(f"Could not load spec for module from file: {script_path}.")
    
    sda_module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(sda_module) 
    print(f"Successfully imported module '{script_filename}' as '{module_name_in_code}'.")
    
except Exception as e:
    print(f"CRITICAL ERROR: Failed to import module '{script_filename}': {e}")
    print("Cannot proceed with analysis.")
    sda_module = None 

--- Script: run_schwab_analysis.py (Schwab Analysis v4) ---
--- Chunk 1: Setup and Module Import ---
Successfully imported module 'schwab_data_analyzer.py' as 'sda'.


In [5]:
print("\n--- Chunk 2: Initialize Client and Fetch All Market Data ---")
print("This step will connect to Schwab API and download/save data. This may take several minutes due to API calls and sleep intervals.")
client = None
if sda_module:
    client = sda_module._initialize_client()

if client: 
    sda_module.fetch_all_market_data(client) 
    print(f"\nINFO: All data fetching attempts from '{script_filename}' are complete.")
    print(f"      Check the '{sda_module.OUTPUT_DIRECTORY}' directory for saved JSON files.")
else:
    print("ERROR: Schwab client not initialized. Skipping data fetching and analysis.")


INFO:Schwabdev:Access token expires in 00H:27M:01S
INFO:Schwabdev:Refresh token expires in 161H:12M:57S
INFO:Schwabdev:Client Initialization Complete



--- Chunk 2: Initialize Client and Fetch All Market Data ---
This step will connect to Schwab API and download/save data. This may take several minutes due to API calls and sleep intervals.
INFO: Schwab client initialized successfully.

=== Fetching data for SPY (ETF) ===

INFO: Fetching Price History for SPY (2 year, daily)...
INFO: Successfully saved: schwab_api_output_options_v4/SPY_price_history_2yr.json
INFO: Price History for SPY (2 year, daily) fetched successfully.

INFO: Fetching Option Expiration Chain for SPY...
INFO: Successfully saved: schwab_api_output_options_v4/SPY_option_exp_chain.json
INFO: Found 32 expiration dates for SPY.

INFO: Fetching Option Chain for SPY (Exp: 2025-05-19, Strikes: 25, Quotes: TRUE)...
ERROR: Exception during Option Chain for SPY (Exp: 2025-05-19, Strikes: 25, Quotes: TRUE): Client.option_chains() got an unexpected keyword argument 'includeQuotes'
INFO: Successfully saved: schwab_api_output_options_v4/SPY_option_chain_20250519_error.json

INFO:

In [ ]:
print("\n--- Chunk 3: Load Processed Price/Volume Data and Generate Plots ---")
dataframes_for_plotting = {}
symbols_and_prefixes_for_plotting = {
    "SPY": ("SPY", "SPY ETF"),
    "SPX_Index": ("SPX_Index", "S&P 500 Index ($SPX.X)"),
    "ES_Future_ES": ("ES_Future_ES", "E-mini S&P Future (ES)"), 
    "ES_Future_SLASHES": ("ES_Future_SLASHES", "E-mini S&P Future (/ES)"),
    "Dow_Jones_Industrial_Average": ("Dow_Jones_Industrial_Average", "Dow Jones ($DJI.X)"),
    "NASDAQ_Composite": ("NASDAQ_Composite", "NASDAQ Comp ($COMPX.X)"),
    "Russell_2000_Small_Cap": ("Russell_2000_Small_Cap", "Russell 2000 ($RUT.X)")
}

if sda_module and client: 
    for df_key, (filename_prefix, display_name_prefix) in symbols_and_prefixes_for_plotting.items():
        price_history_filepath = os.path.join(sda_module.OUTPUT_DIRECTORY, f"{filename_prefix}_price_history_2yr.json")
        error_filepath = os.path.join(sda_module.OUTPUT_DIRECTORY, f"{filename_prefix}_price_history_2yr_error.json")

        if os.path.exists(error_filepath):
            print(f"INFO: Error file found for {display_name_prefix} price history at {error_filepath}, skipping processing.")
            dataframes_for_plotting[df_key] = None
            continue
        
        if os.path.exists(price_history_filepath):
            try:
                with open(price_history_filepath, 'r') as f:
                    history_json = json.load(f)
                
                df = sda_module.get_underlying_price_changes_df(history_json, df_key) 
                dataframes_for_plotting[df_key] = df
                if df is not None:
                    print(f"Successfully loaded and processed price history for: {display_name_prefix}")
                else:
                    pass 
            except Exception as e:
                print(f"ERROR: Could not load/process {price_history_filepath}: {e}")
                dataframes_for_plotting[df_key] = None
        else:
            if not ("ES_Future" in df_key and df_key != "ES_Future_ES"): 
                 print(f"WARNING: Price history file not found for {display_name_prefix} at {price_history_filepath}")
            dataframes_for_plotting[df_key] = None
    
    # Generate plots
    for df_key, (filename_prefix, display_name_prefix) in symbols_and_prefixes_for_plotting.items():
        df_to_plot = dataframes_for_plotting.get(df_key)
        if df_to_plot is not None and not df_to_plot.empty:
            print(f"\nGenerating Price/Volume plot for: {display_name_prefix}...")
            fig = sda_module.plot_price_and_volume(df_to_plot, df_key, display_name_prefix)
            if fig:
                fig.show() 
        elif df_key in dataframes_for_plotting:
            print(f"No data available to plot for {display_name_prefix}.")
else:
    print("Module not loaded or client not initialized. Skipping data loading and plotting.")


--- Chunk 3: Load Processed Price/Volume Data and Generate Plots ---
INFO: Processed price changes and volume (if available) for SPY.
Successfully loaded and processed price history for: SPY ETF
INFO: Processed price changes and volume (if available) for ES_Future_SLASHES.
Successfully loaded and processed price history for: E-mini S&P Future (/ES)

Generating Price/Volume plot for: SPY ETF...


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
print("\n--- Chunk 4: Display Sample Options Data (Textual Summary) ---")
if sda_module and client: 
    sda_module.display_sample_options_data_text("SPY", "SPY ETF")
    sda_module.display_sample_options_data_text("SPX_Index", "S&P 500 Index ($SPX.X)")
    sda_module.display_sample_options_data_text("ES_Future_ES_Options", "E-mini S&P Future (ES attempt)")
    sda_module.display_sample_options_data_text("ES_Future_SLASHES_Options", "E-mini S&P Future (/ES attempt)")
else:
    print("Module not loaded or client not initialized. Skipping options data display.")

print("\n--- Script: run_schwab_analysis.py finished ---")
print("If plots did not appear, check your browser or any error messages above.")